In [27]:
import cv2
import torch
import numpy as np
from PIL import Image
from torch import nn
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation
processor = AutoImageProcessor.from_pretrained("pamixsun/segformer_for_optic_disc_cup_segmentation")
model = SegformerForSemanticSegmentation.from_pretrained("pamixsun/segformer_for_optic_disc_cup_segmentation")
# model.cuda()
from numpy import asarray

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
/home/joheras/.local/lib/python3.10/site-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [35]:
def prediction(image,processor,model):
    # image_new = asarray(Image.open(image))
    # try:
        # image = cv2.imread(image)
        # image_new = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # (h,w)=image_new.shape[:2]
        image_new = asarray(Image.open(image))
        (h,w)=image_new.shape[:2]

        inputs = processor(image_new, return_tensors="pt")
        inputs.to("cuda")
        # inputs = inputs.to(torch.float16)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        upsampled_logits = nn.functional.interpolate(
            logits,
            size=image_new.shape[:2],
            mode="bilinear",
            align_corners=False,
        )

        pred_disc_cup = upsampled_logits.argmax(dim=1)[0].cpu().numpy().astype(np.uint8)

        pred_disc_cup[pred_disc_cup == 2] = 128
        pred_disc_cup[pred_disc_cup == 1] = 255
        img = pred_disc_cup.copy()
        img[img<200]=0
        contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        if (len(contours)  ==0):
            return 0
        scontours = sorted(contours, key=cv2.contourArea, reverse=True)
        _,_,_,h = cv2.boundingRect(scontours[0])
        img = pred_disc_cup
        img[img>200]=0
        img[img>100]=255
        contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        scontours = sorted(contours, key=cv2.contourArea, reverse=True)
        _,_,_,h2 = cv2.boundingRect(scontours[0])
        
        return h2/h
    # except:
    #     return 0

    # return pred_disc_cup
    # disk = np.sum(pred_disc_cup == 1)
    # cup = np.sum(pred_disc_cup == 2)
    # ratio = cup/disk
    # return torch.Tensor([1-ratio,ratio])

In [36]:
model.cuda();

In [37]:
prediction('tmp/image_1.jpg',processor,model)

0.4901185770750988

In [15]:
from tqdm import tqdm
import os
#target_size = (2124, 2056)  # Tamaño deseado de las máscaras
#target_size = (1634, 1634)
for file in tqdm(os.listdir('train/')):
    try:
        
        Image.fromarray(prediction('train/'+file,processor,model)).save('trainMask/' + file.replace('.jpg', '.png'))
        
    except Exception as e:
        print(f"Error processing {file}: {str(e)}")

  8%|▊         | 8223/101425 [3:49:33<46:18:34,  1.79s/it]

Error processing .ipynb_checkpoints: OpenCV(4.7.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



 29%|██▉       | 29722/101425 [13:43:37<28:28:04,  1.43s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 41%|████▏     | 42070/101425 [19:24:41<32:01:54,  1.94s/it]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 88%|████████▊ | 88913/101425 [41:07:36<6:59:34,  2.01s/it] IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--Serve

In [20]:
import json
import tempfile
from pathlib import Path
from pprint import pprint


from PIL import Image

In [21]:
temp_dir = "tmp"

In [38]:
with Image.open('example-algorithm/test/input/stacked-color-fundus-images/images/image.tif') as tiff_image:

    # Iterate through all pages
    print(tiff_image.n_frames)
    for page_num in range(tiff_image.n_frames):
        # Select the current page
        tiff_image.seek(page_num)

        # Define the output file path
        output_path = Path(temp_dir) / f"image_{page_num + 1}.jpg"
        tiff_image.save(output_path, "JPEG")

        # de_stacked_images.append(output_path)

        print(f"De-Stacked {output_path}")

DecompressionBombError: Image size (5210645355 pixels) exceeds limit of 178956970 pixels, could be decompression bomb DOS attack.